In [3]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as plotly
from pandas_profiling import ProfileReport

In [5]:
os.chdir('x:\\Academico\\CUNEF\\TFM\\predictive_maintenance')

In [2]:
index_names = ['engine', 'cycle']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['sensor_{}'.format(i) for i in range(1,22)] 
col_names = index_names + setting_names + sensor_names

In [9]:
train = pd.read_csv('src\\data\\raw\\train_FD001.txt', sep='\s+', header=None, names=col_names)
test = pd.read_csv('src\\data\\raw\\test_FD001.txt', sep='\s+', header=None, names=col_names)
y_test = pd.read_csv(('src\\data\\raw\\RUL_FD001.txt'), sep='\s+', header=None, names=['RUL'])

In [11]:
test_df= test.groupby('engine').agg({'cycle':'max'})
y_test['cycle'] = test_df['cycle']

In [12]:
cycle=[]
for i in range(len(y_test)):
    try:
        cycle.append(y_test['RUL'][i] + y_test['cycle'][i+1])
    except:
        cycle.append(198+20)
        
test_df['cycle']= cycle